#  Machine Learning and Causal Inference - Problem Set 2

By Mathieu Breier, Joaquin Ossa

Date: 19 February 2024

------------------------

### Objective: Calculating the ATE of the RHC dataset, implementing the Double ML technique with cross-fitting

+ Treatment (D): swang1
+ Outcome (Y): death
+ Confounders (X): confounder.yml


#### 1. Train ML model Y~X and calculate residuals using cross-fitting
- Execute cross-validation (train-test split) over a subset of hyperparameters, and choose the one with lower error. You can use functions that simplify the search of optimal parameters, such as caret in (R ) or GridSearchCV in scikit learn


#### 2. Train ML model D~X and calculate residuals using cross-fitting
- Execute cross-validation (train-test split) over a subset of hyperparameters, and choose the one with lower error. You can use functions that simplify the search of optimal parameters, such as caret in (R ) or GridSearchCV in scikit learn


#### 3. Run linear regression Yresiduals~ Dresiduals

Finally, check that the results are similar than the ones DoubleML would give (RHC data ATE estimation with DoubleML (Python).ipynb)

|        | $coef$         | $std\ err$    | $t$           | $P>\|t\|$       | $2.5%$        | $97.5%$       |
|:------:|:------------:|:-----------:|:-----------:|:-----------:|:-----------:|:-----------:|
| swang1 | 0.039061     | 0.013351    | 2.925709    | 0.003437    | 0.012894    | 0.065229    |

----------------



In [1]:
import pandas as pd
import yaml
import numpy as np
from numpy.random import seed
import matplotlib.pyplot as plt
import seaborn as sns

import doubleml as dml

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import mean_squared_error as rmse
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
import random

#warnings
import warnings
warnings.filterwarnings('ignore')

seed(123)

In [ ]:
confounders = [
    "age", "sex", "race", "edu", "income", "ninsclas", "cat1", "das2d3pc", "dnr1", "ca",
    "surv2md1", "aps1", "scoma1", "wtkilo1", "temp1", "meanbp1", "resp1", "hrt1", "pafi1",
    "paco21", "ph1", "wblc1", "hema1", "sod1", "pot1", "crea1", "bili1", "alb1", "resp",
    "card", "neuro", "gastr", "renal", "meta", "hema", "seps", "trauma", "ortho", "cardiohx",
    "chfhx", "dementhx", "psychhx", "chrpulhx", "renalhx", "liverhx", "gibledhx", "malighx",
    "immunhx", "transhx", "amihx"
]

In [3]:
rhc = pd.read_csv("rhc.csv")
with open("confounders.yml", "r") as f:
    confounders = yaml.safe_load(f)

rhc["swang1"] = (rhc["swang1"] == "RHC").astype(int)
rhc["death"] = (rhc["death"] == "Yes").astype(int)
rhc.groupby("swang1")["death"].mean()

rhc_numerical = pd.get_dummies(rhc[['swang1', 'death'] + confounders], dtype=float, drop_first=True)
display(rhc_numerical)


,swang1,death,age,edu,das2d3pc,surv2md1,aps1,scoma1,wtkilo1,temp1,...,resp_Yes,card_Yes,neuro_Yes,gastr_Yes,renal_Yes,meta_Yes,hema_Yes,seps_Yes,trauma_Yes,ortho_Yes
0,0,0,70.25098,12.000000,23.50000,0.640991,46,0,64.69995,38.69531,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,78.17896,12.000000,14.75195,0.755000,50,0,45.69998,38.89844,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1,0,46.09198,14.069916,18.13672,0.317000,82,0,0.00000,36.39844,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,1,75.33197,9.000000,22.92969,0.440979,48,0,54.59998,35.79688,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,1,67.90997,9.945259,21.05078,0.437000,72,41,78.39996,34.79688,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5730,0,0,75.56195,6.000000,14.75195,0.544000,45,9,78.19995,39.79688,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5731,0,1,44.65698,12.000000,23.00000,0.637000,62,0,50.50000,39.09375,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5732,0,1,80.48499,17.000000,17.50000,0.740000,43,0,86.00000,38.00000,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5733,0,1,67.37897,12.000000,16.05859,0.671997,51,9,98.00000,38.79688,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
rhc.head(5)

,Unnamed: 0,cat1,cat2,ca,sadmdte,dschdte,dthdte,lstctdte,death,cardiohx,...,meta,hema,seps,trauma,ortho,adld3p,urin1,race,income,ptid
0,1,COPD,NaN,Yes,11142,11151.0,NaN,11382,0,0,...,No,No,No,No,No,0.0,NaN,white,Under $11k,5
1,2,MOSF w/Sepsis,NaN,No,11799,11844.0,11844.0,11844,1,1,...,No,No,Yes,No,No,NaN,1437.0,white,Under $11k,7
2,3,MOSF w/Malignancy,MOSF w/Sepsis,Yes,12083,12143.0,NaN,12400,0,0,...,No,No,No,No,No,NaN,599.0,white,$25-$50k,9
3,4,ARF,NaN,No,11146,11183.0,11183.0,11182,1,0,...,No,No,No,No,No,NaN,NaN,white,$11-$25k,10
4,5,MOSF w/Sepsis,NaN,No,12035,12037.0,12037.0,12036,1,0,...,No,No,No,No,No,NaN,64.0,white,Under $11k,11


In [5]:
rhc.shape

(5735, 63)

# Analysis
---------

### 1. Regress with DEATH as target


In [6]:
fold1, fold2= next(KFold(n_splits = 2, shuffle = True, random_state = 123).split(rhc_numerical))

print(fold1)
print(fold2)

[   0    1    2 ... 5730 5731 5732]
[   4    6    9 ... 5728 5733 5734]


In [7]:
df = rhc_numerical.drop('swang1', axis = 1).copy()
target = df['death'].copy()

X = df.drop('death', axis=1).copy()
y = target.copy()

In [8]:
# Step 1: Define the number of folds for cross-fitting
n_splits = 2

In [9]:
# Step 2: Define the model
xgb = GradientBoostingClassifier()

In [10]:
# Step 3: Define hyperparameters to search
param_grid = {
    'n_estimators': [150, 200, 250],
    'learning_rate': [ 0.01, 0.1, 0.2],
    'max_depth': [1, 2, 3, 5],
    'min_samples_split': [2, 5, 8],
    'min_samples_leaf': [1, 2, 5, 8]
}

X_train1, X_test1 = X.iloc[fold1], X.iloc[fold2]
y_train1, y_test1 = y.iloc[fold1], y.iloc[fold2]

grid_search1 = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=3, scoring='roc_auc', n_jobs=-1, verbose=3)
grid_search1.fit(X_train1, y_train1)

# SAVING THE BEST MODEL
best_xgb1 = grid_search1.best_estimator_

# TRAINING THE BEST MODEL
best_xgb1.fit(X_train1, y_train1)

# FITTING THE BEST MODEL ON THE TEST SET
train_pred1 = best_xgb1.predict_proba(X_train1)[:, 1]
test_pred1 = best_xgb1.predict_proba(X_test1)[:, 1]

train_auc1 = roc_auc_score(y_train1, train_pred1)
test_auc1 = roc_auc_score(y_test1, test_pred1)


Fitting 3 folds for each of 432 candidates, totalling 1296 fits


[CV 2/3] END learning_rate=0.01, max_depth=1, min_samples_leaf=2, min_samples_split=2, n_estimators=100;, score=0.739 total time=   0.3s
[CV 3/3] END learning_rate=0.01, max_depth=1, min_samples_leaf=2, min_samples_split=2, n_estimators=100;, score=0.711 total time=   0.4s
[CV 1/3] END learning_rate=0.01, max_depth=1, min_samples_leaf=2, min_samples_split=2, n_estimators=100;, score=0.721 total time=   0.4s
[CV 2/3] END learning_rate=0.01, max_depth=1, min_samples_leaf=2, min_samples_split=2, n_estimators=200;, score=0.743 total time=   0.7s
[CV 2/3] END learning_rate=0.01, max_depth=1, min_samples_leaf=2, min_samples_split=5, n_estimators=100;, score=0.739 total time=   0.3s
[CV 1/3] END learning_rate=0.01, max_depth=1, min_samples_leaf=2, min_samples_split=5, n_estimators=100;, score=0.721 total time=   0.4s
[CV 1/3] END learning_rate=0.01, max_depth=1, min_samples_leaf=2, min_samples_split=2, n_estimators=200;, score=0.730 total time=   0.7s
[CV 3/3] END learning_rate=0.01, max_dept

#### DEATH: Results 1

In [11]:
print("Train AUC:", train_auc1)
print("Test AUC:", test_auc1)
print("Best Parameters: ", best_xgb1.get_params())

residuals_Y1 = y_test1 - test_pred1

print("Residuals for Y ~ X on fold 1 as train:", residuals_Y1)

Train AUC: 0.8598245539371627
Test AUC: 0.7694849592232593
Best Parameters:  {'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.1, 'loss': 'log_loss', 'max_depth': 2, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 2, 'min_samples_split': 5, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 200, 'n_iter_no_change': None, 'random_state': None, 'subsample': 1.0, 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}
Residuals for Y ~ X on fold 1 as train: 4       0.056655
6      -0.847274
9      -0.548796
10     -0.088845
11     -0.455699
          ...   
5726    0.400803
5727    0.141984
5728    0.191892
5733    0.332083
5734    0.160336
Name: death, Length: 2868, dtype: float64


In [12]:
X_train2, X_test2 = X.iloc[fold2], X.iloc[fold1]
y_train2, y_test2 = y.iloc[fold2], y.iloc[fold1]

grid_search2 = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=3, scoring='roc_auc', n_jobs=-1, verbose=3)
grid_search2.fit(X_train2, y_train2)

# SAVING THE BEST MODEL
best_xgb2 = grid_search2.best_estimator_

# TRAINING THE BEST MODEL
best_xgb2.fit(X_train2, y_train2)

# FITTING THE BEST MODEL ON THE TEST SET
train_pred2 = best_xgb2.predict_proba(X_train2)[:, 1]
test_pred2 = best_xgb2.predict_proba(X_test2)[:, 1]

train_auc2 = roc_auc_score(y_train2, train_pred2)
test_auc2 = roc_auc_score(y_test2, test_pred2)



Fitting 3 folds for each of 432 candidates, totalling 1296 fits
[CV 3/3] END learning_rate=0.01, max_depth=1, min_samples_leaf=2, min_samples_split=2, n_estimators=100;, score=0.691 total time=   0.3s
[CV 1/3] END learning_rate=0.01, max_depth=1, min_samples_leaf=2, min_samples_split=2, n_estimators=100;, score=0.750 total time=   0.3s
[CV 2/3] END learning_rate=0.01, max_depth=1, min_samples_leaf=2, min_samples_split=2, n_estimators=100;, score=0.734 total time=   0.3s
[CV 2/3] END learning_rate=0.01, max_depth=1, min_samples_leaf=2, min_samples_split=2, n_estimators=200;, score=0.745 total time=   0.6s
[CV 1/3] END learning_rate=0.01, max_depth=1, min_samples_leaf=2, min_samples_split=2, n_estimators=200;, score=0.760 total time=   0.6s
[CV 1/3] END learning_rate=0.01, max_depth=1, min_samples_leaf=2, min_samples_split=5, n_estimators=100;, score=0.750 total time=   0.3s
[CV 2/3] END learning_rate=0.01, max_depth=1, min_samples_leaf=2, min_samples_split=5, n_estimators=100;, score=0.

#### DEATH: Results 2

In [13]:
print("Train AUC:", train_auc2)
print("Test AUC:", test_auc2)
print("Best Parameters: ", best_xgb2.get_params())

residuals_Y2 = y_test2 - test_pred2

print("Residuals for Y ~ X on fold 1 as train:", residuals_Y2)

Train AUC: 0.8078180425021165
Test AUC: 0.7591524008128612
Best Parameters:  {'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.1, 'loss': 'log_loss', 'max_depth': 1, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 2, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 300, 'n_iter_no_change': None, 'random_state': None, 'subsample': 1.0, 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}
Residuals for Y ~ X on fold 1 as train: 0      -0.625514
1       0.394533
2      -0.913642
3       0.224398
5      -0.761661
          ...   
5725   -0.465525
5729   -0.664357
5730   -0.759953
5731    0.474499
5732    0.403208
Name: death, Length: 2867, dtype: float64


### 2. Regress with SWANG1 as target

In [14]:
df = rhc_numerical.drop('death', axis = 1).copy()
#display(df)
target = df['swang1'].copy()

X = df.drop('swang1', axis=1).copy()
y = target.copy()

In [15]:
# Step 1: Define the number of folds for cross-fitting
n_splits = 2
kf = KFold(n_splits=n_splits, shuffle=True, random_state=123)

In [16]:
# Step 2: Define the model
xgb = GradientBoostingClassifier()

In [17]:
# Step 3: Define hyperparameters to search
param_grid = {
    'n_estimators': [150, 200, 250],
    'learning_rate': [ 0.01, 0.1, 0.2],
    'max_depth': [1, 2, 3, 5],
    'min_samples_split': [2, 5, 8],
    'min_samples_leaf': [1, 2, 5, 8]
}

X_train1, X_test1 = X.iloc[fold1], X.iloc[fold2]
y_train1, y_test1 = y.iloc[fold1], y.iloc[fold2]

grid_search1 = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=3, scoring='roc_auc', n_jobs=-1, verbose=3)
grid_search1.fit(X_train1, y_train1)

# SAVING THE BEST MODEL
best_xgb1 = grid_search1.best_estimator_

# TRAINING THE BEST MODEL
best_xgb1.fit(X_train1, y_train1)

# FITTING THE BEST MODEL ON THE TEST SET
train_pred1 = best_xgb1.predict_proba(X_train1)[:, 1]
test_pred1 = best_xgb1.predict_proba(X_test1)[:, 1]

train_auc1 = roc_auc_score(y_train1, train_pred1)
test_auc1 = roc_auc_score(y_test1, test_pred1)



Fitting 3 folds for each of 432 candidates, totalling 1296 fits
[CV 1/3] END learning_rate=0.01, max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=150;, score=0.711 total time=   0.7s
[CV 3/3] END learning_rate=0.01, max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=150;, score=0.687 total time=   0.7s
[CV 2/3] END learning_rate=0.01, max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=150;, score=0.715 total time=   0.7s
[CV 1/3] END learning_rate=0.01, max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=200;, score=0.714 total time=   0.9s
[CV 2/3] END learning_rate=0.01, max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=200;, score=0.728 total time=   0.9s
[CV 3/3] END learning_rate=0.01, max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=200;, score=0.696 total time=   0.8s
[CV 2/3] END learning_rate=0.01, max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=250;, score=0.

#### SWANG1: Results 1

In [18]:
print("Train AUC:", train_auc1)
print("Test AUC:", test_auc1)
print("Best Parameters: ", best_xgb1.get_params())

residuals_D1 = y_test1 - test_pred1

print("Residuals for Y ~ X on fold 1 as train:", residuals_D1)


Train AUC: 0.8433508121243124
Test AUC: 0.7909211117222703
Best Parameters:  {'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.2, 'loss': 'log_loss', 'max_depth': 1, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 8, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 250, 'n_iter_no_change': None, 'random_state': None, 'subsample': 1.0, 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}
Residuals for Y ~ X on fold 1 as train: 4       0.347015
6      -0.385162
9       0.718928
10     -0.575315
11      0.496575
          ...   
5726   -0.365531
5727   -0.658718
5728    0.396563
5733   -0.503409
5734   -0.691209
Name: swang1, Length: 2868, dtype: float64


In [19]:
X_train2, X_test2 = X.iloc[fold2], X.iloc[fold1]
y_train2, y_test2 = y.iloc[fold2], y.iloc[fold1]

grid_search2 = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=3, scoring='roc_auc', n_jobs=-1, verbose=3)
grid_search2.fit(X_train2, y_train2)

# SAVING THE BEST MODEL
best_xgb2 = grid_search2.best_estimator_

# TRAINING THE MODEL
best_xgb2.fit(X_train2, y_train2)

# FITTING THE MODEL ON TRAINING DATA
train_pred2 = best_xgb2.predict_proba(X_train2)[:, 1]
test_pred2 = best_xgb2.predict_proba(X_test2)[:, 1]

train_auc2 = roc_auc_score(y_train2, train_pred2)
test_auc2 = roc_auc_score(y_test2, test_pred2)



Fitting 3 folds for each of 432 candidates, totalling 1296 fits
[CV 2/3] END learning_rate=0.01, max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=150;, score=0.735 total time=   0.5s
[CV 1/3] END learning_rate=0.01, max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=150;, score=0.679 total time=   0.5s
[CV 3/3] END learning_rate=0.01, max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=150;, score=0.722 total time=   0.6s
[CV 3/3] END learning_rate=0.01, max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=200;, score=0.733 total time=   0.7s
[CV 2/3] END learning_rate=0.01, max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=200;, score=0.740 total time=   0.7s
[CV 1/3] END learning_rate=0.01, max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=200;, score=0.681 total time=   0.8s
[CV 1/3] END learning_rate=0.01, max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=250;, score=0.

#### SWANG1: Results 2

In [20]:
print("Train AUC:", train_auc2)
print("Test AUC:", test_auc2)
print("Best Parameters: ", best_xgb2.get_params())

residuals_D2 = y_test2 - test_pred2

print("Residuals for Y ~ X on fold 1 as train:", residuals_D2)

Train AUC: 0.8781646351839315
Test AUC: 0.7989251822483694
Best Parameters:  {'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.1, 'loss': 'log_loss', 'max_depth': 2, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 2, 'min_samples_split': 8, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 150, 'n_iter_no_change': None, 'random_state': None, 'subsample': 1.0, 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}
Residuals for Y ~ X on fold 1 as train: 0      -0.650833
1       0.594001
2       0.583403
3      -0.252596
5      -0.121068
          ...   
5725    0.351910
5729   -0.503322
5730   -0.542411
5731   -0.509351
5732   -0.105393
Name: swang1, Length: 2867, dtype: float64


In [21]:
res_D1 = residuals_D1.copy()
res_Y1 = residuals_Y1.copy()

# Reshape residuals_D for statsmodels compatibility (as a feature matrix)
X_residuals_D1 = sm.add_constant(res_D1.values.reshape(-1, 1))  # Adding a constant for intercept
y_residuals_Y1 = res_Y1.values

# Linear Regression using statsmodels
linear_model_sm1 = sm.OLS(y_residuals_Y1, X_residuals_D1)
results_linear1 = linear_model_sm1.fit()
print(results_linear1.summary())

# Store coefficients and standard errors
coefficients1 = results_linear1.params[1]
std_errors1 = results_linear1.bse[1]

# You can also store other statistics as needed
t_values1 = results_linear1.tvalues[1]
p_values1 = results_linear1.pvalues[1]
conf_int1 = results_linear1.conf_int()[1, :]

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     8.006
Date:                Sat, 24 Feb 2024   Prob (F-statistic):            0.00469
Time:                        19:32:11   Log-Likelihood:                -1614.7
No. Observations:                2868   AIC:                             3233.
Df Residuals:                    2866   BIC:                             3245.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0003      0.008      0.036      0.9

In [22]:
res_D2 = residuals_D2.copy()
res_Y2 = residuals_Y2.copy()

# Reshape residuals_D for statsmodels compatibility (as a feature matrix)
X_residuals_D2 = sm.add_constant(res_D2.values.reshape(-1, 1))  # Adding a constant for intercept
y_residuals_Y2 = res_Y2.values

# Linear Regression using statsmodels
linear_model_sm2 = sm.OLS(y_residuals_Y2, X_residuals_D2)
results_linear2 = linear_model_sm2.fit()
print(results_linear2.summary())

# Store coefficients and standard errors
coefficients2 = results_linear2.params[1]
std_errors2 = results_linear2.bse[1]

# You can also store other statistics as needed
t_values2 = results_linear2.tvalues[1]
p_values2 = results_linear2.pvalues[1]
conf_int2 = results_linear2.conf_int()[1, :]

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     2.713
Date:                Sat, 24 Feb 2024   Prob (F-statistic):             0.0997
Time:                        19:32:11   Log-Likelihood:                -1644.8
No. Observations:                2867   AIC:                             3294.
Df Residuals:                    2865   BIC:                             3305.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0015      0.008     -0.188      0.8

# Results

In [23]:
avg_coeff = np.mean([coefficients1, coefficients2])
avg_std = np.mean([std_errors1, std_errors2])
avg_t = np.mean([t_values1, t_values2])
avg_p = np.mean([p_values1, p_values2])
avg_confmin = np.mean([conf_int1[0], conf_int2[0]])
avg_confmax = np.mean([conf_int1[1], conf_int2[1]])

print("coef:", avg_coeff)
print("std err:", avg_std)
print("t:", avg_t)
print("P > t:", avg_p)
print("2.5", avg_confmin)
print("97.5", avg_confmax)

coef: 0.04232526439611082
std err: 0.018934732198742116
t: 2.2382549578566566
P > t: 0.05217741395537209
2.5 0.005198189143751843
97.5 0.07945233964846979


Finally, check that the results are similar than the ones DoubleML would give (RHC data ATE estimation with DoubleML (Python).ipynb)

|        | $coef$         | $std\ err$    | $t$           | $P>\|t\|$       | $2.5%$        | $97.5%$       |
|:------:|:------------:|:-----------:|:-----------:|:-----------:|:-----------:|:-----------:|
| swang1 | 0.039061     | 0.013351    | 2.925709    | 0.003437    | 0.012894    | 0.065229    |

----------------